In [1]:
%load_ext autoreload
%autoreload 2

import duckdb
import json
import os
import pandas as pd
import pathlib

from utils import *

In [2]:
# Initialize DuckDB connection
conn = duckdb.connect("Piper.db")

In [3]:
conn.execute("DROP TABLE IF EXISTS especies_nativas_endemicas")
conn.execute("""
    CREATE TABLE especies_nativas_endemicas AS
    SELECT * FROM especies_nativas
    UNION ALL
    SELECT * FROM especies_endemicas
""")

In [4]:
df = conn.execute("""
    SELECT scientificName_y, redListCategory, COUNT(DISTINCT(land_regions)) AS region_count
    FROM especies_nativas_endemicas
    GROUP BY scientificName_y, redListCategory
    ORDER BY region_count DESC
""").df()
df.head()

,scientificName_y,redlistCategory,region_count
0,Piper amalago,Least Concern,35
1,Piper aduncum,Least Concern,32
2,Piper hispidum,Least Concern,27
3,Piper tuberculatum,Least Concern,26
4,Piper dilatatum,Least Concern,25


In [5]:
df["enumRedListCategory"] = pd.factorize(df['redlistCategory'])[0]
df.head()

,scientificName_y,redlistCategory,region_count,enumRedListCategory
0,Piper amalago,Least Concern,35,0
1,Piper aduncum,Least Concern,32,0
2,Piper hispidum,Least Concern,27,0
3,Piper tuberculatum,Least Concern,26,0
4,Piper dilatatum,Least Concern,25,0


In [ ]:
df.to_csv("regression.csv", sep=";", quoting=2, index=False, encoding="utf-8")

In [7]:
conn.close()